# Assessing response times from CUAHSI HIS Central AOI web services
- Emilio Mayorga, UW-APL. 2018-4-2
- Run with Python 2.7 using `odm2client` conda environment, the one created for the BiGCZ workshop in Nov. 2017
- Run time includes issuing service request, waiting for response from server, and processing the SOAP response. It was assessed using the [IPYTHON `time` magic function](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)
- **Each API assessment is conducted by issuing a service request over a 1&deg; x 1&deg; AOI box. The center coordinates of this box is specified in cell 9.**
- **For each API assessment, the time to pay attention to is the "Wall time". See results at cells 12, 25 and 31.**
- This notebook also contains a few other explorations of the catalog API responses.

## Code used to calculate the area in $km^2$ of an AOI bounding box
This AOI is specified in lat-lon coordinates as `bbox = (ymin, xmin, ymax, xmax)`

In [1]:
import numpy as np
from shapely.geometry import Polygon

From https://stackoverflow.com/questions/45733838/how-to-calculate-area-of-a-polygon-with-latitude-and-longitude

In [2]:
def to_polygon_vertices(ymin, xmin, ymax, xmax):
    # list of bounding box vertices, to form a closed polygon
    box_points = [(ymin, xmin), (ymax, xmin), (ymax, xmax), (ymin, xmax), (ymin, xmin)]
    return box_points

def switch_to_xy_coordinates(coordinates):
    # returns x,y coordinates in km
    earth_radius = 6371.0  # in km
    lat_dist = np.pi * earth_radius / 180.0

    latitudes, longitudes = zip(*coordinates)
    y = (lat * lat_dist for lat in latitudes)
    x = (lon * lat_dist * np.cos(np.radians(lat)) for lat, lon in zip(latitudes, longitudes))
    return list(zip(x, y))

def compute_polygon_area(coordinates):
    # returns area in km2
    xy_coordinates = switch_to_xy_coordinates(coordinates)
    return Polygon(xy_coordinates).area

## CUAHSI HIS catalog API testing

In [3]:
from collections import OrderedDict
import pandas as pd

In [4]:
import suds.client

## Test WDC HIS Central Series Catalog services

In [5]:
HIS_Central_URL = "http://hiscentral.cuahsi.org/webservices/hiscentral.asmx?WSDL"

In [6]:
client = suds.client.Client(HIS_Central_URL)
cserv = client.service

In [7]:
# Gridded services, which should be considered for exclusion
# determined in advance, manually (there's no other way to identify them)
grid_services = ('GLDAS_NOAH', 'NLDAS_FORA', 'NLDAS_NOAH', 'TRMM_3B42_7')

### Set AOI request parameters

In [8]:
keyword = ''
networkID = ''
start_date = ''
end_date   = ''

sampleMedium, dataType, valueType, generalCategory = '', '', '', ''

In [9]:
# use this scheme to generate a square bounding box centered on the specified lat-lon point
# and with the specified width (in degrees)
width_deg = 1.0

# latctr, lonctr = 40.1, -75.5  # just north of the Schuykil river near Philly
latctr, lonctr = 41.1, -75.5  # 1 deg north of the above PA/DRB center point
# latctr, lonctr = 46.5, -123.0  # About halfway between Olympia, WA and Portland, OR
# latctr, lonctr = 42.0, -93.0  # Iowa
# latctr, lonctr = 30.0, -97.5  # Texas, south of Austin

In [10]:
bbox = (latctr-0.5*width_deg, lonctr-0.5*width_deg, latctr+0.5*width_deg, lonctr+0.5*width_deg)
ymin,xmin,ymax,xmax = bbox

In [11]:
print("BBOX: {0}.  AOI area: {1:.0f} km2".format(bbox, compute_polygon_area(to_polygon_vertices(*bbox))))

BBOX: (40.6, -76.0, 41.6, -75.0).  AOI area: 9317 km2


### TEST 1: GetSeriesCatalogForBox2
Includes extended exploration of the resulting set of series.

In [12]:
%%time
response = cserv.GetSeriesCatalogForBox2(xmin, xmax, ymin, ymax, 
                                         keyword, networkID, 
                                         start_date, end_date)

CPU times: user 50.1 s, sys: 278 ms, total: 50.4 s
Wall time: 55.3 s


In [13]:
type(response), len(response)

(suds.sudsobject.ArrayOfSeriesRecord, 1)

In [14]:
type(response.SeriesRecord), len(response.SeriesRecord)

(list, 16744)

In [15]:
type(response.SeriesRecord[0]), len(response.SeriesRecord[0])

(suds.sudsobject.SeriesRecord, 18)

Import into a DataFrame to facilitate summarization

In [16]:
sr_recs = [OrderedDict(sr) for sr in response.SeriesRecord]
seriesrec_df = pd.DataFrame.from_records(sr_recs, coerce_float=True)

In [17]:
seriesrec_df.head()

,ServCode,ServURL,location,VarCode,VarName,beginDate,endDate,ValueCount,Sitename,latitude,longitude,datatype,valuetype,samplemedium,timeunits,conceptKeyword,genCategory,TimeSupport
0,TRMM_3B42_7,https://hydro1.gesdisc.eosdis.nasa.gov/daac-bi...,TRMM_3B42_7:X418-Y364,TRMM_3B42_7:TRMM_3B42.007:precipitation,precipitation,1997-12-31T00:00:00Z,2018-03-19T06:34:23Z,43800,X418-Y364 of TRMM Multi-Satellite Precipitatio...,41.125,-75.375,Incremental,Derived Value,Precipitation,hour,Precipitation,Climate,1
1,TRMM_3B42_7,https://hydro1.gesdisc.eosdis.nasa.gov/daac-bi...,TRMM_3B42_7:X419-Y362,TRMM_3B42_7:TRMM_3B42.007:precipitation,precipitation,1997-12-31T00:00:00Z,2018-03-19T06:34:23Z,43800,X419-Y362 of TRMM Multi-Satellite Precipitatio...,40.625,-75.125,Incremental,Derived Value,Precipitation,hour,Precipitation,Climate,1
2,TRMM_3B42_7,https://hydro1.gesdisc.eosdis.nasa.gov/daac-bi...,TRMM_3B42_7:X416-Y364,TRMM_3B42_7:TRMM_3B42.007:precipitation,precipitation,1997-12-31T00:00:00Z,2018-03-19T06:34:23Z,43800,X416-Y364 of TRMM Multi-Satellite Precipitatio...,41.125,-75.875,Incremental,Derived Value,Precipitation,hour,Precipitation,Climate,1
3,TRMM_3B42_7,https://hydro1.gesdisc.eosdis.nasa.gov/daac-bi...,TRMM_3B42_7:X419-Y363,TRMM_3B42_7:TRMM_3B42.007:precipitation,precipitation,1997-12-31T00:00:00Z,2018-03-19T06:34:23Z,43800,X419-Y363 of TRMM Multi-Satellite Precipitatio...,40.875,-75.125,Incremental,Derived Value,Precipitation,hour,Precipitation,Climate,1
4,TRMM_3B42_7,https://hydro1.gesdisc.eosdis.nasa.gov/daac-bi...,TRMM_3B42_7:X417-Y364,TRMM_3B42_7:TRMM_3B42.007:precipitation,precipitation,1997-12-31T00:00:00Z,2018-03-19T06:34:23Z,43800,X417-Y364 of TRMM Multi-Satellite Precipitatio...,41.125,-75.625,Incremental,Derived Value,Precipitation,hour,Precipitation,Climate,1


Number of series by service

In [18]:
seriesrec_df.ServCode.value_counts()

ShaleNetworkODM    12152
NWISGW              2279
CocoRaHs             694
GHCN                 383
NLDAS_FORA           320
NLDAS_NOAH           256
NWISDV               226
GLDAS_NOAH           208
NWISUV               123
UNH_Snow              50
EnviroDIY             32
TRMM_3B42_7           16
MOPEX                  5
Name: ServCode, dtype: int64

In [19]:
seriesrec_df.valuetype.value_counts()

Field Observation          10432
Unknown                     4992
Derived Value                607
Model Simulation Result      448
Sample                       265
Name: valuetype, dtype: int64

In [20]:
seriesrec_df.genCategory.value_counts()

Water Quality      12151
Hydrology           3468
Climate             1093
Water quality         24
Instrumentation        8
Name: genCategory, dtype: int64

In [21]:
# I'm filtering out "*DAS_NOAH* service codes, b/c they're from gridded models
# (results represent each cell) and therefore overwhelm the response
series_array_nongrid = [series for series in response.SeriesRecord 
                        if series.ServCode not in grid_services]

len(series_array_nongrid)

15944

### TEST 2: GetSeriesMetadataCountOrData

`GetSeriesMetadataCountOrData` arguments, in order:   
getData, getFacetOnCV,   
xmin, xmax, ymin, ymax,   
sampleMedium, dataType, valueType, generalCategory,   
conceptKeyword, networkIDs,   
beginDate, endDate

First use the API to request only the number of series that match the search criteria (`getData = False`)

In [22]:
%%time
response = cserv.GetSeriesMetadataCountOrData(False, False, 
                                              xmin, xmax, ymin, ymax, 
                                              sampleMedium, dataType, valueType, generalCategory, 
                                              keyword, networkID, 
                                              start_date, end_date)

CPU times: user 1.99 ms, sys: 7.86 ms, total: 9.85 ms
Wall time: 183 ms


In [23]:
response

(GetSeriesCountOrData){
   nseries = 16744
 }

In [24]:
response.nseries

16744L

Now request the actual series records (`getData = True`)

In [25]:
%%time
response = cserv.GetSeriesMetadataCountOrData(True, False, 
                                              xmin, xmax, ymin, ymax, 
                                              sampleMedium, dataType, valueType, generalCategory, 
                                              keyword, networkID, 
                                              start_date, end_date)

CPU times: user 1min 47s, sys: 659 ms, total: 1min 47s
Wall time: 1min 50s


In [26]:
type(response), len(response)

(suds.sudsobject.GetSeriesCountOrData, 2)

In [27]:
response.nseries, response[0]

(16744L, 16744L)

In [28]:
type(response.series), len(response.series)

(suds.sudsobject.ArrayOfSeriesRecordFull2, 1)

In [29]:
type(response.series.SeriesRecordFull2), len(response.series.SeriesRecordFull2)

(list, 16744)

In [30]:
response.series.SeriesRecordFull2[8]

(SeriesRecordFull2){
   ServCode = "TRMM_3B42_7"
   ServURL = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/his/1.0/trmm_3b42.cgi?WSDL"
   location = "TRMM_3B42_7:X416-Y363"
   VarCode = "TRMM_3B42_7:TRMM_3B42.007:precipitation"
   VarName = "precipitation"
   beginDate = "1997-12-31T00:00:00Z"
   endDate = "2018-03-19T06:35:17Z"
   ValueCount = "43800"
   Sitename = "X416-Y363 of TRMM Multi-Satellite Precipitation Analysis (TMPA-RT)"
   latitude = "40.875"
   longitude = "-75.875"
   datatype = "Incremental"
   valuetype = "Derived Value"
   samplemedium = "Precipitation"
   timeunits = "hour"
   conceptKeyword = "Precipitation"
   genCategory = "Climate"
   TimeSupport = "1"
   QCLID = "2"
   QCLDesc = "Derived products"
   Organization = "NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC)"
   TimeUnitAbbrev = "hr"
   IsRegular = "true"
   Speciation = "Not Applicable"
   SourceOrg = "NASA Goddard Earth Sciences (GES) Data and Information Services Center

### TEST 3: GetSeriesCatalogForBox3

In [31]:
%%time
# Provides more query parameters, and a much richer response, vs GetSeriesCatalogForBox2
response = cserv.GetSeriesCatalogForBox3(xmin, xmax, ymin, ymax, 
                                         sampleMedium, dataType, valueType,
                                         keyword, networkID, 
                                         start_date, end_date)

CPU times: user 1min 53s, sys: 596 ms, total: 1min 54s
Wall time: 2min


In [32]:
response.SeriesRecordFull[2]

(SeriesRecordFull){
   ServCode = "TRMM_3B42_7"
   ServURL = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/his/1.0/trmm_3b42.cgi?WSDL"
   location = "TRMM_3B42_7:X416-Y364"
   VarCode = "TRMM_3B42_7:TRMM_3B42.007:precipitation"
   VarName = "precipitation"
   beginDate = "1997-12-31T00:00:00Z"
   endDate = "2018-03-19T06:37:12Z"
   ValueCount = 43800
   Sitename = "X416-Y364 of TRMM Multi-Satellite Precipitation Analysis (TMPA-RT)"
   latitude = 41.125
   longitude = -75.875
   datatype = "Incremental"
   valuetype = "Derived Value"
   samplemedium = "Precipitation"
   timeunits = "hour"
   conceptKeyword = "Precipitation"
   genCategory = "Climate"
   TimeSupport = "1"
   SeriesCode = "479|X416-Y364|TRMM_3B42.007:precipitation|1|1|2"
   QCLID = "2"
   QCLDesc = "Derived products"
   Organization = "NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC)"
   TimeUnitAbbrev = "hr"
   TimeUnits = "hour"
   IsRegular = "true"
   Speciation = "Not Applicable"
   So